Открываем файл с текстом:

In [16]:
with open ('korosteleva_carmarthen.txt') as f:
  data = f.read()

Мы знаем, что истории внутри книги разделены тремя звёздочками. А еще обрежем первые 43 символа -- это название книги и автор. Обрежем также примечания к тексту -- они начинаются после фразы "Август 2000 г."

In [17]:
chapters = data[44:].split('* * *')
chapters[-1] = chapters[-1].split('Август 2000 г')[0]

# Извлечение имен
Теперь для каждой истории извлечем все имена, которые там встречаются

Для этого воспользуемся лингвистической библиотекой **Natasha**. Наташа -- безумно умная библиотека, которая в числе прочего умеет извлекать из текста именованные сущности -- названия людей, географический объектов и так далее.


> Наташа -- это нейросетевая модель, обученная на новостях с размеченными именованными сущностями; поэтому она отлично справляется с новостями, но и с другими жанрами она справляется тоже неплохо.


*(Хотя с нашими валлийскими и не только именами она помучается..))))*

### Сначала технические шаги:
* устанавливаем библиотеку
* импортируем нужные блоки


In [ ]:
!pip install natasha pymorphy2

In [ ]:
from natasha import (
    Segmenter,
    MorphVocab,

    NewsEmbedding,
    NewsMorphTagger,
    NewsNERTagger,

    PER,
    NamesExtractor,

    Doc
)

segmenter = Segmenter()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
ner_tagger = NewsNERTagger(emb)

morph_vocab = MorphVocab()
names_extractor = NamesExtractor(morph_vocab)

Посмотрим, как Наташа извлекает имена

In [ ]:
text = 'Афина приехала в подмосковное Пущино на школу Сова. Она знакомится с библиотекой Наташа'
doc = Doc(text)

doc.segment(segmenter)
doc.tag_morph(morph_tagger)

doc.tag_ner(ner_tagger)
for span in doc.spans:
    print(span)

DocSpan(stop=5, type='PER', text='Афина', tokens=[...])
DocSpan(start=30, stop=36, type='LOC', text='Пущино', tokens=[...])
DocSpan(start=46, stop=50, type='LOC', text='Сова', tokens=[...])
DocSpan(start=81, stop=87, type='PER', text='Наташа', tokens=[...])


Дальше определим функцию, которая берет главу, и извлекает из нее названия персонажей

In [ ]:
from tqdm import tqdm

In [28]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [29]:
def NER(text):
    """
    Функция для извлечения именованных сущностей
    """
    doc = Doc(text) # обрезаю часть про тг-канал и ошибки, которая есть в каждой новости

    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    doc.tag_ner(ner_tagger)

    ner = []

    for s in doc.spans:
        tokens = s.tokens
        lemmas = []
        for t in tokens:
            l = morph.parse(t.text)[0].normal_form
            lemmas.append(l)
        n = (" ".join(lemmas), s.type)
        ner.append(n)

    return ner


def get_names(book):
    '''
    функция, которая перебирает главы и записывает, сколько раз персонажи встретились вместе в одной главе
    input: list глав
    output: словарь имен и словарь связей
    '''
    persons = {} # все имена
    connect = {} # связи

    for art in tqdm(book):
        nfacts = NER(art) # извлекаем имена
        # Так как один участник может упоминаться несколько раз, строим список с единственными упоминаниями.
        nam = [fact[0].split(" ")[-1] for fact in nfacts if fact[1] == 'PER']
        snam = list(set(nam))
        # Пробрасываем связи между людьми. Главное - не писать сколько раз человек связан между собой.
        for n in snam:
            persons[n] = persons.get(n, 0) + 1
            pers = connect.get(n, {})
            for n2 in snam:
                if n != n2:
                    pers[n2] = pers.get(n2, 0)+1
            connect[n] = pers
    return persons, connect

Извлечем связи из книги:

In [30]:
names, connections = get_names(chapters)

100%|██████████| 211/211 [00:49<00:00,  4.24it/s]


### Теперь давайте введем несколько ограничений:
* оставим только самые частотные имена -- такие, которые встречаются в целом по книге больше 10 раз (`names[n2] > 10`)
* оставим только персонажей, "взаимодействовавших" (то есть встречавшихся в одной главе) больше пяти раз (`connections[n][n2] > 5`)
* удалим всякие перлы Наташи: например, она лемматизирует Змейка как *змейка*, а Керидвен -- как *керидвена*; а еще считает, что скобка -- это имя собственное (`n2 != ')' and n2 != 'змейка' and n2 != 'керидвена'`)



In [31]:
pers2 = {
    n: {
        n2: connections[n][n2]
        for n2 in connections[n].keys()
        if connections[n][n2] > 5 and names[n2] > 10 and n2 != ')' and n2 != 'змейка' and n2 != 'керидвена'
    }
    for n in connections.keys()
    if names[n] > 10 and n != ')' and n != 'змейка' and n != 'керидвена'
}


In [32]:
pers2

{'курой': {'карх': 17,
  'морган-ап-керричь': 6,
  'лютгарда': 6,
  'змейк': 28,
  'кехт': 13,
  'орбилия': 7,
  'мерлина': 27,
  'мэлдун': 8,
  'финтан': 13,
  'гвидион': 22,
  'оуэн': 6,
  'хлодвиг': 6,
  'ллевелиса': 22,
  'коллен': 6,
  'керидвено': 8,
  'тарквиния': 10,
  'кромвелеть': 7,
  'сюань-цзан': 6},
 'карх': {'курой': 17,
  'морган-ап-керричь': 7,
  'змейк': 16,
  'кехт': 11,
  'орбилия': 7,
  'мерлина': 24,
  'мэлдун': 9,
  'финтан': 12,
  'гвидион': 16,
  'ллевелиса': 18,
  'бервин': 10,
  'оуэн': 10,
  'финтана': 6,
  'тарквиния': 7},
 'морган-ап-керричь': {'курой': 6,
  'карх': 7,
  'змейк': 8,
  'мерлина': 10,
  'мэлдун': 6},
 'рианнона': {'змейк': 14,
  'кехт': 21,
  'мерлина': 15,
  'финтан': 6,
  'гвидион': 19,
  'ллевелиса': 13,
  'морвидда': 7,
  'керидвено': 6,
  'коллен': 6,
  'диана': 7},
 'лютгарда': {'курой': 6,
  'змейк': 8,
  'мерлина': 9,
  'гвидион': 7,
  'ллевелиса': 6},
 'змейк': {'курой': 28,
  'карх': 16,
  'морган-ап-керричь': 8,
  'рианнона': 14,


И запишем все в CSV-файл, чтобы потом скормить его Gephi.

In [33]:
import csv

with open('character_interactions.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Source', 'Target', 'Weight'])  # такие заголовки требуются для gephi

    for person1, interactions in pers2.items():
        for person2, weight in interactions.items():
            writer.writerow([person1, person2, weight])